In [4]:
'https://www.kaggle.com/code/vbmokin/used-cars-price-prediction-by-15-models/input'
import os
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
#import pandas_profiling as pp

root = os.path.dirname(os.path.abspath("."))
sys.path.append(root)

import predikit as pk

In [7]:
train0=pd.read_csv("./sample_data/craigslistVehicles.csv")
train0.head(5)

,url,city,city_url,price,year,manufacturer,make,condition,cylinders,fuel,...,transmission,VIN,drive,size,type,paint_color,image_url,desc,lat,long
0,https://grandrapids.craigslist.org/cto/d/hasti...,"grand rapids, MI",https://grandrapids.craigslist.org,1500,2006.0,cadillac,cts,good,6 cylinders,gas,...,automatic,NaN,rwd,mid-size,coupe,blue,https://images.craigslist.org/00K0K_a9CZoZg2U8...,"2006 CtS Leather, Runs and drives Good.236k mil",42.643000,-85.293700
1,https://grandrapids.craigslist.org/cto/d/grand...,"grand rapids, MI",https://grandrapids.craigslist.org,8900,2009.0,lincoln,mkx,NaN,NaN,gas,...,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00a0a_9B4kPBDIWd...,"Selling our loaded 2009 Lincoln MKX with 119,0...",42.973700,-85.726500
2,https://grandrapids.craigslist.org/ctd/d/chesa...,"grand rapids, MI",https://grandrapids.craigslist.org,7995,2010.0,cadillac,srx premium collection,NaN,NaN,gas,...,automatic,3GYFNCEYXAS552363,NaN,NaN,NaN,NaN,https://images.craigslist.org/00X0X_8i0VRuk7Cv...,WE HAVE OVER 400 VEHICLES IN STOCK!\n\n View O...,43.186723,-84.163862
3,https://grandrapids.craigslist.org/ctd/d/chesa...,"grand rapids, MI",https://grandrapids.craigslist.org,6995,2007.0,NaN,hummer h3 4dr 4wd suv,NaN,NaN,gas,...,automatic,5GTDN13E478107380,NaN,NaN,NaN,NaN,https://images.craigslist.org/00b0b_ahkmUzr4cE...,WE HAVE OVER 400 VEHICLES IN STOCK!\n\n View O...,43.186723,-84.163862
4,https://grandrapids.craigslist.org/ctd/d/caled...,"grand rapids, MI",https://grandrapids.craigslist.org,20990,2010.0,ram,2500,excellent,6 cylinders,diesel,...,automatic,3D7UT2CL4AG113236,4wd,NaN,NaN,white,https://images.craigslist.org/00505_3DHY0kFrgb...,Great looking 2010 Ram 2500 ST w/6.7L 24V I6 4...,42.783714,-85.506777


In [8]:
drop_columns = ['url', 'city', 'city_url', 'make', 'title_status', 'VIN', 'size', 'image_url', 'desc', 'lat','long']
train0 = train0.drop(columns = drop_columns)

In [9]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525839 entries, 0 to 525838
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         525839 non-null  int64  
 1   year          524399 non-null  float64
 2   manufacturer  501260 non-null  object 
 3   condition     279881 non-null  object 
 4   cylinders     315439 non-null  object 
 5   fuel          521544 non-null  object 
 6   odometer      427248 non-null  float64
 7   transmission  521572 non-null  object 
 8   drive         374475 non-null  object 
 9   type          376906 non-null  object 
 10  paint_color   354306 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 44.1+ MB


In [10]:
train0 = train0.dropna()
train0.head(5)

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
0,1500,2006.0,cadillac,good,6 cylinders,gas,236000.0,automatic,rwd,coupe,blue
5,4950,2010.0,subaru,good,4 cylinders,gas,253000.0,automatic,4wd,sedan,white
6,6850,2007.0,gmc,good,8 cylinders,gas,254000.0,automatic,4wd,wagon,black
7,7995,2007.0,lexus,excellent,6 cylinders,gas,146111.0,automatic,fwd,sedan,white
8,4995,2011.0,hyundai,excellent,4 cylinders,gas,115048.0,automatic,fwd,sedan,blue


In [11]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = train0.columns.values.tolist()
for col in features:
    if train0[col].dtype in numerics: continue
    categorical_columns.append(col)
# Encoding categorical features
for col in categorical_columns:
    if col in train0.columns:
        le = LabelEncoder()
        le.fit(list(train0[col].astype(str).values))
        train0[col] = le.transform(list(train0[col].astype(str).values))

In [12]:
train0['year'] = (train0['year']-1900).astype(int)
train0['odometer'] = train0['odometer'].astype(int)

train0.head(10)

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
0,1500,106,6,2,5,2,236000,0,2,3,1
5,4950,110,36,2,3,2,253000,0,0,9,10
6,6850,107,14,2,6,2,254000,0,0,12,0
7,7995,107,23,0,5,2,146111,0,1,9,10
8,4995,111,17,0,3,2,115048,0,1,9,1
9,12995,113,37,0,3,2,72936,0,1,4,10
12,8000,110,3,2,5,2,150000,0,0,9,5
18,6995,111,7,0,5,2,102000,0,1,9,1
21,39000,117,7,0,6,2,43300,0,2,9,5
24,8900,112,33,2,6,2,112000,0,2,8,10


In [13]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144903 entries, 0 to 525836
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   price         144903 non-null  int64
 1   year          144903 non-null  int32
 2   manufacturer  144903 non-null  int32
 3   condition     144903 non-null  int32
 4   cylinders     144903 non-null  int32
 5   fuel          144903 non-null  int32
 6   odometer      144903 non-null  int32
 7   transmission  144903 non-null  int32
 8   drive         144903 non-null  int32
 9   type          144903 non-null  int32
 10  paint_color   144903 non-null  int32
dtypes: int32(10), int64(1)
memory usage: 7.7 MB


In [14]:
train0['price'].value_counts()

price
0        6755
1        1743
4500     1603
3500     1593
2500     1528
         ... 
12761       1
5445        1
25470       1
26810       1
3297        1
Name: count, Length: 6578, dtype: int64

In [15]:
train0 = train0[train0['price'] > 1000]
train0 = train0[train0['price'] < 40000]
# Rounded ['odometer'] to 5000
train0['odometer'] = train0['odometer'] // 5000
train0 = train0[train0['year'] > 110]
train0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61500 entries, 8 to 525835
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   price         61500 non-null  int64
 1   year          61500 non-null  int32
 2   manufacturer  61500 non-null  int32
 3   condition     61500 non-null  int32
 4   cylinders     61500 non-null  int32
 5   fuel          61500 non-null  int32
 6   odometer      61500 non-null  int32
 7   transmission  61500 non-null  int32
 8   drive         61500 non-null  int32
 9   type          61500 non-null  int32
 10  paint_color   61500 non-null  int32
dtypes: int32(10), int64(1)
memory usage: 3.3 MB


In [16]:
target_name = 'price'
train_target0 = train0[target_name]
train0 = train0.drop([target_name], axis=1)
# Synthesis test0 from train0
train0, test0, train_target0, test_target0 = train_test_split(train0, train_target0, test_size=0.2, random_state=0)
# For boosting model
train0b = train0
train_target0b = train_target0
# Synthesis valid as test for selection models
trainb, testb, targetb, target_testb = train_test_split(train0b, train_target0b, test_size=valid_part, random_state=0)
#For models from Sklearn
scaler = StandardScaler()
train0 = pd.DataFrame(scaler.fit_transform(train0), columns = train0.columns)

In [18]:
# Synthesis valid as test for selection models
train, test, target, target_test = train_test_split(train0, train_target0, test_size=valid_part, random_state=0)

In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34440 entries, 27131 to 2732
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          34440 non-null  float64
 1   manufacturer  34440 non-null  float64
 2   condition     34440 non-null  float64
 3   cylinders     34440 non-null  float64
 4   fuel          34440 non-null  float64
 5   odometer      34440 non-null  float64
 6   transmission  34440 non-null  float64
 7   drive         34440 non-null  float64
 8   type          34440 non-null  float64
 9   paint_color   34440 non-null  float64
dtypes: float64(10)
memory usage: 2.9 MB


In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14760 entries, 44567 to 19061
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          14760 non-null  float64
 1   manufacturer  14760 non-null  float64
 2   condition     14760 non-null  float64
 3   cylinders     14760 non-null  float64
 4   fuel          14760 non-null  float64
 5   odometer      14760 non-null  float64
 6   transmission  14760 non-null  float64
 7   drive         14760 non-null  float64
 8   type          14760 non-null  float64
 9   paint_color   14760 non-null  float64
dtypes: float64(10)
memory usage: 1.2 MB


In [28]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score

# Define the lists to store accuracy values
acc_train_r2 = []
acc_test_r2 = []
acc_train_d = []
acc_test_d = []
acc_train_rmse = []
acc_test_rmse = []

def acc_d(y_meas, y_pred):
    # Relative error between predicted y_pred and measured y_meas values
    return mean_absolute_error(y_meas, y_pred)*len(y_meas)/sum(abs(y_meas))

def acc_rmse(y_meas, y_pred):
    # RMSE between predicted y_pred and measured y_meas values
    return (mean_squared_error(y_meas, y_pred))**0.5

def acc_boosting_model(num,model,train,test,num_iteration=0):
    # Calculation of accuracy of boosting model by different metrics
    
    global acc_train_r2, acc_test_r2, acc_train_d, acc_test_d, acc_train_rmse, acc_test_rmse
    
    if num_iteration > 0:
        ytrain = model.predict(train, num_iteration = num_iteration)  
        ytest = model.predict(test, num_iteration = num_iteration)
    else:
        ytrain = model.predict(train)  
        ytest = model.predict(test)

    print('target = ', targetb[:5].values)
    print('ytrain = ', ytrain[:5])

    acc_train_r2_num = round(r2_score(targetb, ytrain) * 100, 2)
    print('acc(r2_score) for train =', acc_train_r2_num)   
    acc_train_r2.insert(num, acc_train_r2_num)

    acc_train_d_num = round(acc_d(targetb, ytrain) * 100, 2)
    print('acc(relative error) for train =', acc_train_d_num)   
    acc_train_d.insert(num, acc_train_d_num)

    acc_train_rmse_num = round(acc_rmse(targetb, ytrain) * 100, 2)
    print('acc(rmse) for train =', acc_train_rmse_num)   
    acc_train_rmse.insert(num, acc_train_rmse_num)

    print('target_test =', target_testb[:5].values)
    print('ytest =', ytest[:5])
    
    acc_test_r2_num = round(r2_score(target_testb, ytest) * 100, 2)
    print('acc(r2_score) for test =', acc_test_r2_num)
    acc_test_r2.insert(num, acc_test_r2_num)
    
    acc_test_d_num = round(acc_d(target_testb, ytest) * 100, 2)
    print('acc(relative error) for test =', acc_test_d_num)
    acc_test_d.insert(num, acc_test_d_num)
    
    acc_test_rmse_num = round(acc_rmse(target_testb, ytest) * 100, 2)
    print('acc(rmse) for test =', acc_test_rmse_num)
    acc_test_rmse.insert(num, acc_test_rmse_num)

def acc_model(num,model,train,test):
    # Calculation of accuracy of model акщь Sklearn by different metrics   
  
    global acc_train_r2, acc_test_r2, acc_train_d, acc_test_d, acc_train_rmse, acc_test_rmse
    
    ytrain = model.predict(train)  
    ytest = model.predict(test)

    print('target = ', target[:5].values)
    print('ytrain = ', ytrain[:5])

    acc_train_r2_num = round(r2_score(target, ytrain) * 100, 2)
    print('acc(r2_score) for train =', acc_train_r2_num)   
    acc_train_r2.insert(num, acc_train_r2_num)

    acc_train_d_num = round(acc_d(target, ytrain) * 100, 2)
    print('acc(relative error) for train =', acc_train_d_num)   
    acc_train_d.insert(num, acc_train_d_num)

    acc_train_rmse_num = round(acc_rmse(target, ytrain) * 100, 2)
    print('acc(rmse) for train =', acc_train_rmse_num)   
    acc_train_rmse.insert(num, acc_train_rmse_num)

    print('target_test =', target_test[:5].values)
    print('ytest =', ytest[:5])
    
    acc_test_r2_num = round(r2_score(target_test, ytest) * 100, 2)
    print('acc(r2_score) for test =', acc_test_r2_num)
    acc_test_r2.insert(num, acc_test_r2_num)
    
    acc_test_d_num = round(acc_d(target_test, ytest) * 100, 2)
    print('acc(relative error) for test =', acc_test_d_num)
    acc_test_d.insert(num, acc_test_d_num)
    
    acc_test_rmse_num = round(acc_rmse(target_test, ytest) * 100, 2)
    print('acc(rmse) for test =', acc_test_rmse_num)
    acc_test_rmse.insert(num, acc_test_rmse_num)


In [29]:
from predikit import Regressor
Ada_Boost = Regressor(strategy='AdaBoostRegressor')
Ada_Boost.fit(train, target)
acc_model(13,Ada_Boost,train,test)

target =  [14888  8995 16990  5600  5490]
ytrain =  [16470.30544678 11600.44421326 17046.1402195  13168.14504505
 11096.18626765]
acc(r2_score) for train = 61.25
acc(relative error) for train = 26.13
acc(rmse) for train = 537556.99
target_test = [38000 31550  5900 13500 35000]
ytest = [21818.9362411  24440.52721239 10526.17681289 20153.43101803
 26805.14738019]
acc(r2_score) for test = 61.43
acc(relative error) for test = 25.86
acc(rmse) for test = 535400.58
